In [1]:
import numpy as np
import pandas as pd

## A Multiply indexed Series

### The bad way
(use python dictionary which keys are tuple as index)

In [2]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [3]:
pop[('New York',2000):('Texas', 2000)]

(New York, 2000)    18976457
(New York, 2010)    19378102
(Texas, 2000)       20851820
dtype: int64

If we need to select all values from 2010, we have to access the tuple index first, and then access data from 2010.  
This is very inflexible

In [4]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

### The better way: Pandas MultiIndex

In [5]:
# we can create a multi-index from the tuples as follows

In [6]:
print('index1 :', index)
index = pd.MultiIndex.from_tuples(index)
print('index2 :', index)

index1 : [('California', 2000), ('California', 2010), ('New York', 2000), ('New York', 2010), ('Texas', 2000), ('Texas', 2010)]
index2 : MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           codes=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])


In [7]:
# re-index our series
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [8]:
pop[1]

37253956

Some entries are missing in the first column: in this multi-index representation, any blank entry indicates the same value as the line above it 

In [9]:
pop[:,2010]


California    37253956
New York      19378102
Texas         25145561
dtype: int64

In [10]:
pop['California']

2000    33871648
2010    37253956
dtype: int64

### MultiIndex as extra dimension

The `unstack()` method will quickly convert a multiply indexed `Series` into a conventionally indexed `DataFrame`

In [11]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


Naturally, the `stack()` method provides the opposite operation

In [12]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Using multi-index we will have much more flexiblity in the types of data we can represent.  
As multi-index can be used to represent two-dimensional data within a one-dimensional `Series`, we can use it to represent data of three or more dimensions in a `Series` or `DataFrame`

In [13]:
pop_df = pd.DataFrame({'total':pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df     

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

The ufuncs and other functionality also work with hierachical index

In [14]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


## Creating MultiIndex

The more straightforward way to construct a multiply indexed `Series` or `DataFrame` is to simply pass a list of index arrays

In [15]:
df = pd.DataFrame(
    np.random.rand(4, 2),
    index=[['a', 'a', 'b', 'b'], ['0', '1', '0', '1']],
    columns=['data1', 'data2'])
df

data1     data2
a 0  0.219944  0.230192
  1  0.363897  0.251871
b 0  0.070257  0.222828
  1  0.448896  0.833094

We can also pass a dictionary with appropriate tuples as kes, Pandas will automatically recognize this and use a `MultiIndex` by default

In [16]:
data = {
    ('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102
}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

### Explicit MultiIndex constructors

In [17]:
# from a list of arrays
pd.MultiIndex.from_arrays([['a','a','b','b'], ['0','1','0','1']])


MultiIndex(levels=[['a', 'b'], ['0', '1']],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [18]:
# from a list tuple
pd.MultiIndex.from_tuples([('a',0),('a',1),('b',0),('b',1)])

MultiIndex(levels=[['a', 'b'], [0, 1]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [19]:
# from a Cartesian product of single indices
pd.MultiIndex.from_product([['a','b'],[0,1]])

MultiIndex(levels=[['a', 'b'], [0, 1]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [20]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

### MultiIndex level names

In [21]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### MultiIndex for columns

Rows and columns are complete symmetric, so the columns can have multiple levels as well

In [22]:
# Hierachical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37
data

array([[49. , 37. , 37. , 37.5, 32. , 37.5],
       [32. , 39. , 42. , 36.2, 36. , 37. ],
       [30. , 38.3, 33. , 37.2, 26. , 35.8],
       [43. , 36.3, 33. , 36.1, 31. , 37.4]])

In [23]:
# create the DataFrame
health_data = pd.DataFrame(data, index=index,
                           columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      49.0  37.0  37.0  37.5  32.0  37.5
     2      32.0  39.0  42.0  36.2  36.0  37.0
2014 1      30.0  38.3  33.0  37.2  26.0  35.8
     2      43.0  36.3  33.0  36.1  31.0  37.4

In [24]:
health_data.loc[::2,'Guido':'Sue']

subject    Guido         Sue      
type          HR  Temp    HR  Temp
year visit                        
2013 1      37.0  37.5  32.0  37.5
2014 1      33.0  37.2  26.0  35.8

## Indexing and Slicing a MultiIndex


### Multiply indexed Series

In [25]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [26]:
# We can access single elements by indexing with multiple terms
pop['California',2010]

37253956

In [27]:
# Partial indexing
# The result is another `Series`, with the lower-level indices maintained
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [28]:
pop.loc['California':'New York', 2000]

state       year
California  2000    33871648
New York    2000    18976457
dtype: int64

In [29]:
pop.iloc[0:2]

state       year
California  2000    33871648
            2010    37253956
dtype: int64

In [30]:
pop[:,2010]

state
California    37253956
New York      19378102
Texas         25145561
dtype: int64

In [31]:
pop[pop>22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [32]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### Multiply indexed DataFrames

In [33]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      49.0  37.0  37.0  37.5  32.0  37.5
     2      32.0  39.0  42.0  36.2  36.0  37.0
2014 1      30.0  38.3  33.0  37.2  26.0  35.8
     2      43.0  36.3  33.0  36.1  31.0  37.4

Remember that columns are **primary** in a `DataFrame`, and the syntax used for multiply indexed `Series` applies to the columns

In [34]:
health_data['Guido', 'HR']

year  visit
2013  1        37.0
      2        42.0
2014  1        33.0
      2        33.0
Name: (Guido, HR), dtype: float64

In [35]:
# xxx.iloc[rows, columns]
health_data.iloc[:2, :3]

subject      Bob       Guido
type          HR  Temp    HR
year visit                  
2013 1      49.0  37.0  37.0
     2      32.0  39.0  42.0

These indexers provide an array-like view of the underlying two-dimensional data,but each individual index in `loc` or `iloc` can be passed a tuple of multiple indices

In [36]:
health_data.loc[:,('Guido', 'HR')]

year  visit
2013  1        37.0
      2        42.0
2014  1        33.0
      2        33.0
Name: (Guido, HR), dtype: float64

In [37]:
health_data.loc[(2013, 1),('Guido', 'HR')]

37.0

In [38]:
health_data.loc[2013:,(['Guido','Bob'], 'HR')]

subject      Bob Guido
type          HR    HR
year visit            
2013 1      49.0  37.0
     2      32.0  42.0
2014 1      30.0  33.0
     2      43.0  33.0

Working with slices within these index tuples is not especially convenient; trying to create a slice within a tuple will lead to a syntax error

In [39]:
health_data.loc[(:,1),(:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-39-186aa6537cbc>, line 1)

Getting around this problem by using Pandas's `IndexSlice` object

In [42]:
idx = pd.IndexSlice
health_data.loc[idx[:,::2], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,49.0,37.0,32.0
2014,1,30.0,33.0,26.0


## Rearranging Multi-Indices

### Sorted and unsorted indices

Many of the `MultiIndex` slicing operations will `fail` if the index is not sorted

Let's start by creating some simple multiply indexed data where the indices are **not lexographically** sorted

If we try to take a partial slice of this index, it will result in an `error`

In [43]:
# we can see the index has not benn sorted
index = pd.MultiIndex.from_product([['a','c','b'],[1,2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names=['char', 'int']
data

char  int
a     1      0.309068
      2      0.389323
c     1      0.362703
      2      0.521367
b     1      0.528231
      2      0.649981
dtype: float64

In [45]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)


<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


For various reasons, partial slicess and other similar operations require the leveks in the `MultiIndex` to be **in sorted order**  
Pandas provides a number of convenience routines to perform this type of sorting.  
e.g.:  
`sort_index()` and `sortlevel()` methods of the `DataFrame`.

In [47]:
data = data.sort_index()
data

char  int
a     1      0.309068
      2      0.389323
b     1      0.528231
      2      0.649981
c     1      0.362703
      2      0.521367
dtype: float64

In [48]:
data['a':'b']

char  int
a     1      0.309068
      2      0.389323
b     1      0.528231
      2      0.649981
dtype: float64

### Stacking and unstacking indices

In [50]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

There are some options specifying the level.


In [52]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [53]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


### Index setting and resetting

In [55]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [58]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [59]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

## Data Aggregations on Multi-Indices

We've previously seen that Pandas has built-in data aggregation methods, such as ``mean()``, ``sum()``, and ``max()``.
For hierarchically indexed data, these can be passed a ``level`` parameter that controls which subset of the data the aggregate is computed on.

For example, let's return to our health data:

In [60]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      49.0  37.0  37.0  37.5  32.0  37.5
     2      32.0  39.0  42.0  36.2  36.0  37.0
2014 1      30.0  38.3  33.0  37.2  26.0  35.8
     2      43.0  36.3  33.0  36.1  31.0  37.4

If we'd like to average-out the measurements in the two visit each year. We can do this by **naming the index level**

In [61]:
data_mean = health_data.mean(level='year')
data_mean

subject   Bob       Guido          Sue       
type       HR  Temp    HR   Temp    HR   Temp
year                                         
2013     40.5  38.0  39.5  36.85  34.0  37.25
2014     36.5  37.3  33.0  36.65  28.5  36.60

In [ ]:
health_data.mean(level='visit')

By further making use of the `aixs` keyword, we can take the mean among levels on the columns as well

In [ ]:
health_data.sum(axis=1, level='type')

In [ ]:
health_data.sum(axis=1, level='subject')